In [2]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('data/publications.csv')

print("="*80)
print("COMPLETE DATA PROFILING - COUNTRY RESEARCH PERFORMANCE")
print("="*80)

# ============================================================================
# 1. BASIC STRUCTURE
# ============================================================================
print("\n📊 DATASET STRUCTURE:")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Time Period: {df['year'].min()} - {df['year'].max()} ({df['year'].max() - df['year'].min() + 1} years)")
print(f"Unique Countries: {df['Name'].nunique()}")
print(f"\nCountries: {', '.join(sorted(df['Name'].unique()))}")

# ============================================================================
# 2. DATA QUALITY
# ============================================================================
print("\n\n" + "="*80)
print("DATA QUALITY:")
print(f"Missing Values: {df.isnull().sum().sum()} (Perfect!)" if df.isnull().sum().sum() == 0 else df.isnull().sum())
print(f"Duplicates: {df.duplicated().sum()}")

# ============================================================================
# 3. KEY STATISTICS
# ============================================================================
print("\n\n" + "="*80)
print("KEY PERFORMANCE METRICS:")

metrics = {
    'Total Research Output': df['Web of Science Documents'].sum(),
    'Total Citations': df['Times Cited'].sum(),
    'Avg Citations per Paper': df['Times Cited'].sum() / df['Web of Science Documents'].sum(),
    'Avg CNCI (baseline=1.0)': df['Category Normalized Citation Impact'].mean(),
    'Records with CNCI > 1.0': f"{(df['Category Normalized Citation Impact'] > 1.0).sum()} ({(df['Category Normalized Citation Impact'] > 1.0).mean()*100:.1f}%)",
    'Avg % Papers in Top 10%': df['% Documents in Top 10%'].mean(),
    'Avg % Papers in Top 1%': df['% Documents in Top 1%'].mean(),
}

for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key:.<40} {value:>15.2f}")
    else:
        print(f"{key:.<40} {str(value):>15}")

# ============================================================================
# 4. TOP PERFORMERS (by different metrics)
# ============================================================================
print("\n\n" + "="*80)
print("TOP 5 COUNTRIES BY DIFFERENT METRICS:")

# Aggregate by country
country_agg = df.groupby('Name').agg({
    'Web of Science Documents': 'sum',
    'Times Cited': 'sum',
    'Category Normalized Citation Impact': 'mean',
    '% Documents in Top 1%': 'mean',
    '% Documents in Top 10%': 'mean',
    'Collab-CNCI': 'mean'
}).round(2)

print("\n🥇 By Total Citations:")
top_citations = country_agg.nlargest(5, 'Times Cited')['Times Cited']
for country, value in top_citations.items():
    print(f"   {country}: {value:,.0f}")

print("\n💫 By Avg CNCI (Quality):")
top_cnci = country_agg.nlargest(5, 'Category Normalized Citation Impact')['Category Normalized Citation Impact']
for country, value in top_cnci.items():
    print(f"   {country}: {value:.3f}")

print("\n🏆 By % in Top 1% (Excellence):")
top_excellence = country_agg.nlargest(5, '% Documents in Top 1%')['% Documents in Top 1%']
for country, value in top_excellence.items():
    print(f"   {country}: {value:.2f}%")

# ============================================================================
# 5. TEMPORAL TRENDS
# ============================================================================
print("\n\n" + "="*80)
print("TEMPORAL TRENDS:")

yearly = df.groupby('year').agg({
    'Category Normalized Citation Impact': 'mean',
    'Web of Science Documents': 'sum',
    '% Documents in Top 10%': 'mean'
}).round(2)

print("\n📈 CNCI Over Time (First vs Last 3 years):")
print(f"2003-2005 avg: {yearly.loc[2003:2005, 'Category Normalized Citation Impact'].mean():.3f}")
print(f"2023-2025 avg: {yearly.loc[2023:2025, 'Category Normalized Citation Impact'].mean():.3f}")
trend = yearly.loc[2023:2025, 'Category Normalized Citation Impact'].mean() - yearly.loc[2003:2005, 'Category Normalized Citation Impact'].mean()
print(f"Change: {'+' if trend > 0 else ''}{trend:.3f}")

print("\n📚 Output Trend:")
print(f"2003-2005 total docs: {yearly.loc[2003:2005, 'Web of Science Documents'].sum():,.0f}")
print(f"2023-2025 total docs: {yearly.loc[2023:2025, 'Web of Science Documents'].sum():,.0f}")
growth = ((yearly.loc[2023:2025, 'Web of Science Documents'].sum() / yearly.loc[2003:2005, 'Web of Science Documents'].sum()) - 1) * 100
print(f"Growth: {growth:.1f}%")

# ============================================================================
# 6. COLLABORATION EFFECT
# ============================================================================
print("\n\n" + "="*80)
print("COLLABORATION ADVANTAGE:")

collab_advantage = df['Collab-CNCI'].mean() - df['Category Normalized Citation Impact'].mean()
print(f"Avg Regular CNCI: {df['Category Normalized Citation Impact'].mean():.3f}")
print(f"Avg Collab-CNCI: {df['Collab-CNCI'].mean():.3f}")
print(f"Collaboration Advantage: {'+' if collab_advantage > 0 else ''}{collab_advantage:.3f}")

positive_collab = (df['Collab-CNCI'] > df['Category Normalized Citation Impact']).sum()
print(f"Records where collaboration helps: {positive_collab} ({positive_collab/len(df)*100:.1f}%)")

# ============================================================================
# 7. INTERESTING PATTERNS
# ============================================================================
print("\n\n" + "="*80)
print("INTERESTING PATTERNS:")

# Quality vs Quantity
print("\n📊 Quality vs Quantity:")
high_output = df[df['Web of Science Documents'] > df['Web of Science Documents'].median()]
low_output = df[df['Web of Science Documents'] <= df['Web of Science Documents'].median()]
print(f"High-output countries avg CNCI: {high_output['Category Normalized Citation Impact'].mean():.3f}")
print(f"Low-output countries avg CNCI: {low_output['Category Normalized Citation Impact'].mean():.3f}")

# Excellence concentration
print("\n🎯 Excellence Distribution:")
high_excellence = df[df['% Documents in Top 1%'] > 2.0]
print(f"Records with >2% in Top 1%: {len(high_excellence)} ({len(high_excellence)/len(df)*100:.1f}%)")
print(f"They account for {high_excellence['Web of Science Documents'].sum()/df['Web of Science Documents'].sum()*100:.1f}% of total output")

# Most consistent performers
print("\n⭐ Most Consistent Countries (by CNCI std dev):")
consistency = df.groupby('Name')['Category Normalized Citation Impact'].agg(['mean', 'std', 'count'])
consistency = consistency[consistency['count'] >= 5]  # At least 5 year records
consistency = consistency.sort_values('std')
print(consistency.head(5)[['mean', 'std']])

# ============================================================================
# 8. SUGGESTED STORY ANGLES
# ============================================================================
print("\n\n" + "="*80)
print("💡 SUGGESTED STORY ANGLES:")
print("="*80)

print("\n1. 'The Quality Paradox'")
print(f"   → {(df['Category Normalized Citation Impact'] < 1.0).sum()} records ({(df['Category Normalized Citation Impact'] < 1.0).mean()*100:.0f}%) have below-average impact")
print(f"   → But top performers achieve CNCI of {df['Category Normalized Citation Impact'].max():.2f}")

print("\n2. 'Collaboration Multiplier Effect'")
print(f"   → Collaboration provides {collab_advantage:.3f} CNCI advantage on average")

print("\n3. 'Evolution Over Two Decades'")
print(f"   → Research output grew {growth:.0f}% from 2003-05 to 2023-25")
print(f"   → But average quality {'improved' if trend > 0 else 'declined'} by {abs(trend):.3f}")

print("\n4. 'The Excellence Gap'")
print(f"   → Top countries have {top_excellence.iloc[0]:.2f}% in Top 1%")
print(f"   → While average is only {df['% Documents in Top 1%'].mean():.2f}%")

print("\n\n" + "="*80)
print("✅ COPY THE OUTPUT ABOVE AND SHARE WITH ME!")
print("="*80)



COMPLETE DATA PROFILING - COUNTRY RESEARCH PERFORMANCE

📊 DATASET STRUCTURE:
Shape: 1000 rows × 12 columns
Time Period: 2003 - 2025 (23 years)
Unique Countries: 17

Countries: AUSTRALIA, BRAZIL, CANADA, CHINA, ENGLAND, FRANCE, GERMANY, INDIA, ITALY, JAPAN, NETHERLANDS, SOUTH KOREA, SPAIN, SWEDEN, SWITZERLAND, UNITED KINGDOM, USA


DATA QUALITY:
Missing Values: 0 (Perfect!)
Duplicates: 0


KEY PERFORMANCE METRICS:
Total Research Output...................        14861699
Total Citations.........................      1296496955
Avg Citations per Paper.................           87.24
Avg CNCI (baseline=1.0).................            1.29
Records with CNCI > 1.0.................     855 (85.5%)
Avg % Papers in Top 10%.................           17.59
Avg % Papers in Top 1%..................            1.77


TOP 5 COUNTRIES BY DIFFERENT METRICS:

🥇 By Total Citations:
   SPAIN: 98,635,709
   SWITZERLAND: 92,412,437
   BRAZIL: 92,375,787
   CHINA: 85,742,760
   ITALY: 85,321,548

💫 By Avg